# 🏬 **TASACIÓN DE INMUEBLES** 🏢

Esta información de coordenadas, es la continuación como anexos de la base de datos proveniente de *zonaprop.com.ar*.

### 6. Visualización de Coordenadas 🏹

#### **6.1) Librerías utilizadas**

Detalle de los paquetes y librerías utilizadas.

In [ ]:
#-------- PAQUETES INSTALADAS --------#

!pip install --upgrade pip
!pip install --upgrade Pillow
!pip install --upgrade geodatasets



#-------- LIBRERÍAS --------#

# -> Librerías de uso básico
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas
import geodatasets

# -> Librerías Complementarias
import sys
import math
from pathlib import Path

Se toma la misma dirección de la database desarrollada de la sección **1. Curaduría de datos**

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1A-4oAsQCneD4HDJRTQgO0WTRQtW0JRnw/edit?usp=sharing&ouid=109033079849202436389&rtpof=true&sd=true"
file_id=url.split('/')[-2]
path ='https://drive.google.com/uc?id=' + file_id

Carga y visualización de la database.

In [ ]:
# -> Lectura del archivo
df = pd.read_excel(path)

# -> Verificación de separador
df.columns

Index(['idPost', 'Precio', 'title', 'Description', 'direccion', 'barrio',
       'm2total', 'm2cubierto', 'Banos', 'Dormitorios', 'Antiguedad',
       'Estrenar', 'Cochera', 'Estado', 'Luminoso', 'coordenadas.lat',
       'coordenadas.lng', 'terminos_relevantes', 'Type', 'desc_relevante',
       'Suite', 'Pileta', 'Patio', 'Oportunidad', 'Reciclar', 'Country',
       'Exc_ubi'],
      dtype='object')

#### **6.2) Preparación de la database para el Plot**

Verificación aleatoria de carga del Database

In [ ]:
df.sample(3)

,idPost,Precio,title,Description,direccion,barrio,m2total,m2cubierto,Banos,Dormitorios,...,terminos_relevantes,Type,desc_relevante,Suite,Pileta,Patio,Oportunidad,Reciclar,Country,Exc_ubi
5814,48824772,190.0,Casa de 3 Dorm. con Pileta y Cochera (3) - 450...,✧ Casa de 1 sola planta ✧ 3 dormitorios ✧ Livi...,"Salto 261, Juniors, Córdoba",Juniors,450,290,3,3,...,Casa Dorm Pileta Cochera Juniors,Casa,casa planta dormitorios living comedor hogar c...,False,True,False,False,False,False,False
1331,44466262,150.0,Venta Casa B° Colinas del Cerro,Venta CASA de 3 dormitorios en Barrio Colinas ...,"Argibay 4000, Colinas del Cerro, Córdoba",Colinas del Cerro,420,300,3,3,...,Venta Casa Colinas Cerro,Casa,venta casa dormitorios barrio colinas cerro pr...,True,True,True,False,False,False,False
6576,46920637,490.0,Exc. Casa en Vta en El Bosque,"CASA EN EXCELENTE ESTADO DE CONSERVACION, UBIC...","s/n s/n, Córdoba, Córdoba",Córdoba,1500,280,3,2,...,Exc Casa Vta Bosque,Casa,casa excelente conservacion ubicada cerca ingr...,True,False,False,False,False,False,False


Las Features a utilizar son **direccion**, **coordenadas.lat**, **coordenadas.lng**.

A continuación, se extraen para visualizar la información que contiene

In [ ]:
df[['direccion', 'coordenadas.lat', 'coordenadas.lng']]

,direccion,coordenadas.lat,coordenadas.lng
0,"Calle Lote / N° 5, DOCTA, Córdoba",-3.142008e+16,-6.418877e+16
1,"Country Cañuelas Lote / N° 0, Cañuelas Country...",-3.147486e+16,-6.416392e+16
2,"Portales Del Sur (calle Huiliches), Parque Atl...",-3.145716e+16,-6.420963e+16
3,"CaÑuelas Lote / N° 0, Cañuelas Country Golf, C...",-3.147486e+16,-6.416392e+16
4,"9 De Julio 3300, Alto Alberdi, Córdoba",-3.140223e+16,-6.422527e+16
...,...,...,...
7017,"Tupungato 2500, Parque Capital, Córdoba",-3.143145e+16,-6.422153e+16
7018,"Albert Brucce Sabin 6100, Argüello, Córdoba",-3.134698e+16,-6.427047e+16
7019,"Tte. Melchor Escola 5300, Jorge Newbery, Córdoba",-3.135450e+16,-6.418322e+16
7020,"Macha 3100, Villa Rivera Indarte, Córdoba",-3.131410e+16,-6.429459e+16


##### ***6.2.1) Cambio nombre de variables***

Para una mejor lectura de las variables de almacenamiento a utilizar con la **librería Folium**, se reemplazarán los nombres de las columnas **direccion**, **coordenadas.lat**, **coordenadas.lng**.

In [ ]:
df_coordenadas = {'direccion': 'name',
                  'coordenadas.lat': 'latitud',
                  'coordenadas.lng': 'longitud'}

df = df.rename(columns=df_coordenadas)

Verificación del cambio realizado:

In [ ]:
df[['name', 'latitud', 'longitud']]

,name,latitud,longitud
0,"Calle Lote / N° 5, DOCTA, Córdoba",-3.142008e+16,-6.418877e+16
1,"Country Cañuelas Lote / N° 0, Cañuelas Country...",-3.147486e+16,-6.416392e+16
2,"Portales Del Sur (calle Huiliches), Parque Atl...",-3.145716e+16,-6.420963e+16
3,"CaÑuelas Lote / N° 0, Cañuelas Country Golf, C...",-3.147486e+16,-6.416392e+16
4,"9 De Julio 3300, Alto Alberdi, Córdoba",-3.140223e+16,-6.422527e+16
...,...,...,...
7017,"Tupungato 2500, Parque Capital, Córdoba",-3.143145e+16,-6.422153e+16
7018,"Albert Brucce Sabin 6100, Argüello, Córdoba",-3.134698e+16,-6.427047e+16
7019,"Tte. Melchor Escola 5300, Jorge Newbery, Córdoba",-3.135450e+16,-6.418322e+16
7020,"Macha 3100, Villa Rivera Indarte, Córdoba",-3.131410e+16,-6.429459e+16


##### ***6.2.2) Revisión y Filling de NaN´s***

A continuación se realiza un *conteo* de NaN´s, y posteriormente se hará el proceso de *Filling* para no presentar errores en el plot.

In [ ]:
print('NaN´s latitud: {}'.format(df['latitud'].isnull().sum()))
print('NaN´s longitud: {}'.format(df['longitud'].isnull().sum()))
print('NaN´s name: {}'.format(df['name'].isnull().sum()))

NaN´s latitud: 58
NaN´s longitud: 58
NaN´s name: 34


In [ ]:
df['latitud'] = df['latitud'].fillna(0)
df['longitud'] = df['longitud'].fillna(0)
df['name'] = df['name'].fillna(0)

Verificación de NaN´s:

In [ ]:
print('Verificación de NaN´s Latitud: {}'.format(df['latitud'].isnull().sum()))
print('Verificación de NaN´s Longitud: {}'.format(df['longitud'].isnull().sum()))
print('Verificación de NaN´s Name: {}'.format(df['name'].isnull().sum()))

Verificación de NaN´s Latitud: 0
Verificación de NaN´s Longitud: 0
Verificación de NaN´s Name: 0


##### ***6.2.3) Redimensión de las unidades***

Dadas la magnitud de los registros de **latitud**, **longitud**, es necesario llevar acabo una redimensión de las unidades para que la librería pueda realizar el plot.

In [ ]:
df_divisor = 1000000000000000
df_coord = df[['latitud', 'longitud']].div(df_divisor, axis=0)

# Visualización aleatoria de 3 filas
df_coord.sample(3)

,latitud,longitud
6189,-31.393529,-64.146980
4330,-31.387544,-64.238222
5891,-31.392823,-64.148682


##### ***6.2.4) Muestreo aleatorio de coordenadas a graficar***

Debido al tamaño de la database, para este caso únicamente se visualizarán 500 coordenadas.

In [ ]:
# Muestreo de Database a plotear en Folium
df_coord_sample = df_coord.sample(500)

# Visualización aleatoria de las 500 coordenadas
df_coord_sample

,latitud,longitud
3192,-31.455099,-64.237864
5436,-31.461714,-64.186251
490,-31.343408,-64.260048
80,-31.344347,-64.225646
6859,-31.443759,-64.168941
...,...,...
4809,-31.352101,-64.224211
5607,-31.376518,-64.247963
1003,-31.322395,-64.286036
1447,-31.456915,-64.170556


#### **6.3) Plot de coordenadas**

Para este proyecto se utilizó la **Librería Folium**.

Toda la información de su uso se encuentra alojada en el siguiente enlace:
👉🏻 https://python-visualization.github.io/folium/latest/

In [ ]:
#-------- LIBRERÍA --------#

import folium

##### ***6.3.1) Carga de variables e iteración***

Habiendo definido el *Cambio de nombre de variables (2.2.1)*, y el *Muestreo aleatorio de coordenadas a graficar (2.2.4)*, se procede a pasar dichas variables al código requerido en la librería para crear el mapa.

In [ ]:
map = folium.Map(location=[df_coord_sample.latitud.mean(), df_coord_sample.longitud.mean()], zoom_start=12, control_scale=True)

In [ ]:
for index, location_info in df_coord_sample.iterrows():
    folium.Marker([location_info["latitud"], location_info["longitud"]]).add_to(map)

##### ***6.3.2) Plot aleatorio de la Database***

In [ ]:
# Displaying the map
map

#### **6.4) Conclusión**

Nuestro objetivo con este anexo, es poder acercar de manera visual a cualquier cliente (***Bien sea local, o alguna persona que sea ajena a la región***), una idea de la ubicación de algunos de los inmuebles involucrados en la Database.

Consideramos a este como un importante valor agregado tanto para el área involucrada en ventas, o bien al proceso individual como herramienta de negociación.